# Image Processing with NumPy

- **Student Info:** 21CLC05 - 21127135 - Diep Huu Phuc
- **Github Repository: https://github.com/kru01/ImageProcessing_NumPy**

## Contents

- [Import libraries](#import-libraries)
- [1, 2, 3. Brightness, Contrast, Flipping](#1-2-3-brightness-contrast-flipping)
- [4. Grayscale and Sepia](#4-grayscale-and-sepia)
- [5. Blurring and Sharpening](#5-blurring-and-sharpening)
- [6, 7. Center and Circle cropping](#6-7-center-and-circle-cropping)
- [Extra. Cross Ellipses cropping](#extra-cross-ellipses-cropping)
- [Miscellanous functions](#miscellaneous-functions)
- [Main program](#main-program-handling-interfaces-inputs-and-outputs)
- [References](#references)
- [Past implementations](#past-implementations)

## Import libraries

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

## 1, 2, 3. Brightness, Contrast, Flipping
1. [Algorithm to modify brightness for RGB image?, 2014-06-03](https://stackoverflow.com/a/24022126) - [Patrick](https://stackoverflow.com/users/774398/patrick)

$$ brightness\_channel = channel + (255f * brightness\_factor) $$

2. [IMAGE PROCESSING ALGORITHMS PART 5: CONTRAST ADJUSTMENT, 2015](https://www.dfstudios.co.uk/articles/programming/image-programming-algorithms/image-processing-algorithms-part-5-contrast-adjustment/) - [Francis G. Loch](https://twitter.com/francisloch)

$$ F = \frac{259*(255+C)}{255*(259-C)} $$
$$ channel = F * (channel - 128) + 128 $$

In [ ]:
def adjust_brightness(img:np.ndarray, factor:float=0.5):
    '''
    > img: np.ndarray with shape=(height, width, num_channels)
        Original image as an array
    > factor: float representing how much the brightness will be adjusted
        E.g., -0.5 darkens by 50%, 0.35 brightens by 35%

    < new_img: np.ndarray with shape=(height, width, num_channels)
    '''
    return np.clip(img + 255.0 * factor, 0, 255).astype(np.uint8)

def adjust_contrast(img:np.ndarray, factor:float=0.5):
    '''
    > img: np.ndarray with shape=(height, width, num_channels)
        Original image as an array
    > factor: float representing how much the contrast will be adjusted
        E.g., -0.5 adjusts by -127=int(-0.5*255), 0.35 adjusts by 89=int(0.35*255)

    < new_img: np.ndarray with shape=(height, width, num_channels)
    '''
    contrast = int(np.clip(255 * factor, -255, 255))
    factor = 259 * (255 + contrast) / (255 * (259 - contrast))
    return np.clip(factor * (img - 128.0) + 128, 0, 255).astype(np.uint8)

def flip(img:np.ndarray, axis:int=0):
    '''
    > img: np.ndarray with shape=(height, width, num_channels)
        Original image as an array
    > axis: int representing the direction to flip the image
        0 for vertical, 1 for horizontal

    < new_img: np.ndarray with shape=(height, width, num_channels)
    '''
    return np.flip(img, np.clip(int(axis), 0, 1))

## 4. Grayscale and Sepia

4. [How to Convert an RGB Image to a Grayscale, 2023-06-19](https://www.baeldung.com/cs/convert-rgb-to-grayscale) - [Panagiotis Antoniadi](https://www.baeldung.com/cs/author/panagiotisantoniadis)

   - Average method: $$ grayscale = \frac{R+G+B}{3} $$
   - Luminosity method: $$ grayscale = 0.3R + 0.59G + 0.11B $$

5. [How to convert a color image into sepia image, 2014-01-27](https://dyclassroom.com/image-processing-project/how-to-convert-a-color-image-into-sepia-image) - [Yusuf Shakeel](https://www.youtube.com/@yusufshakeel)

$$ tr = 0.393R + 0.769G + 0.189B \\ tg = 0.349R + 0.686G + 0.168B \\ tb = 0.272R + 0.534G + 0.131B $$

In [ ]:
def rgb_to_grayscale(img:np.ndarray, method:int=1):
    '''
    > img: np.ndarray with shape=(height, width, num_channels)
        Original image as an array
    > method: int representing how to grayscale the image
        0 for average (mean) method, 1 for luminosity (weighted) method

    < new_img: np.ndarray with shape=(height, width, num_channels)
    '''
    methods = ['average', 'luminosity']
    method = np.clip(int(method), 0, len(methods) - 1)
    if method == 0: return np.mean(img, axis=2).astype(np.uint8)
    if method == 1:
        rgb_weights = np.array([0.3, 0.59, 0.11])
        return np.matmul(img, rgb_weights).astype(np.uint8)

def rgb_to_sepia(img:np.ndarray):
    '''
    > img: np.ndarray with shape=(height, width, num_channels)
        Original image as an array
    < new_img: np.ndarray with shape=(height, width, num_channels)
    '''
    rgb_weights = np.array([[0.393, 0.769, 0.189],
                            [0.349, 0.686, 0.168],
                            [0.272, 0.534, 0.131]])
    Rs = np.matmul(img, rgb_weights[0])
    Gs = np.matmul(img, rgb_weights[1])
    Bs = np.matmul(img, rgb_weights[2])
    return np.clip(np.dstack((Rs, Gs, Bs)), 0, 255).astype(np.uint8)

## 5. Blurring and Sharpening

- [ImageBlurring, 2023-05-06](https://github.com/pooyakalahroodi/ImageBlurring) - [pooya kalahroodi](https://github.com/pooyakalahroodi)

In [ ]:
def convolve(img:np.ndarray, kernel:np.ndarray):
    '''
    > img: np.ndarray with shape=(height, width, num_channels)
        Original image as an array
    > kernel: np.ndarray with shape=(3, 3)
        Convolution matrix, or mask, to convolve with the image

    < new_img: np.ndarray with shape=(height, width, num_channels)
    '''
    if img.ndim == 2: padded_img = np.zeros((img.shape[0] + 2, img.shape[1] + 2))
    else:
        padded_img = np.zeros((img.shape[0] + 2, img.shape[1] + 2, img.shape[2]))
        kernel = kernel[..., None]
    padded_img[1:-1, 1:-1] = img
    new_img = np.zeros(img.shape)

    if img.ndim == 2:
        for row in range(img.shape[0]):
            for col in range(img.shape[1]):
                new_img[row, col] = np.multiply(kernel, padded_img[row:row+3, col:col+3]).sum()
        return new_img.astype(np.uint8)

    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            rgbs_of_rows = np.multiply(kernel, padded_img[row:row+3, col:col+3]).sum(axis=0)
            new_img[row, col] = np.sum(rgbs_of_rows, axis=0)
    return new_img.astype(np.uint8)

def gaussian_blur_3x3(img:np.ndarray):
    '''
    > img: np.ndarray with shape=(height, width, num_channels)
        Original image as an array
    < new_img: np.ndarray with shape=(height, width, num_channels)
    '''
    kernel = 1 / 16 * np.array([[1, 2, 1],
                               [2, 4, 2],
                               [1, 2, 1]])
    return convolve(img, kernel)

def sharpen(img:np.ndarray):
    '''
    > img: np.ndarray with shape=(height, width, num_channels)
        Original image as an array
    < new_img: np.ndarray with shape=(height, width, num_channels)
    '''
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    return convolve(img, kernel)

## 6, 7. Center and Circle cropping

7. [How can I create a circular mask for a numpy array?, 2020-02-07](https://stackoverflow.com/a/44874588) - [alkasm](https://stackoverflow.com/users/5087436/alkasm)


In [ ]:
def center_crop(img:np.ndarray, dim:tuple|float=0.5):
    '''
    > img: np.ndarray with shape=(height, width, num_channels)
        Original image as an array
    > dim: tuple (height, width) representing the dimension of the cropped image
        float representing the ratio between the cropped and the original image

    < new_img: np.ndarray with shape=(dim[0], dim[1], num_channels)
    '''
    if type(dim) is tuple: cropped_height, cropped_width = dim
    else: cropped_height, cropped_width = img.shape[0] * dim, img.shape[1] * dim
    cropped_height = np.clip(cropped_height, 2, None)
    cropped_width = np.clip(cropped_width, 2, None)

    center = (int(img.shape[0] / 2), int(img.shape[1] / 2))
    ver_offset, hor_offset = int(cropped_height / 2), int(cropped_width / 2)
    return img[center[0] - ver_offset : center[0] + ver_offset,
               center[1] - hor_offset : center[1] + hor_offset]

def circle_crop(img:np.ndarray, radius:int|float=0, center:tuple=None):
    '''
    > img: np.ndarray with shape=(height, width, num_channels)
        Original image as an array
    > radius: int (rad > 1) representing the radius of the cropped image
        float (0 < rad <= 1) representing ratio between the cropped and the original image
    > center: tuple (y, x) representing the coordinate of the center of the cropped image

    < new_img: np.ndarray with shape=(dim[0], dim[1], num_channels)
    '''
    if not center: center = (int(img.shape[0] / 2), int(img.shape[1] / 2))
    if not radius or radius < 0:
        radius = min(center[0], center[1], img.shape[0] - center[0], img.shape[1] - center[1])
    elif 0 < radius and radius <= 1:
        radius = min(img.shape[0] * radius, img.shape[1] * radius)
    
    Ys, Xs = np.ogrid[:img.shape[0], :img.shape[1]]
    distances_to_center = np.sqrt((Ys - center[0])**2 + (Xs - center[1])**2)
    mask = distances_to_center <= radius
    new_img = img.copy()
    new_img[~mask] = 0
    return new_img

## Extra. Cross Ellipses cropping

- Will be made publicly available after 2023-July-30.

## Miscellaneous functions

- These functions are solely for handling inputs and outputs, they **definitely are NOT** the main focus of this notebook.

In [ ]:
def plot_and_save(filename:str, og_img:list|np.ndarray, out_imgs:list, titles:list, save_files:list=None):
    if len(titles) < 3:
        out_imgs = [og_img] + out_imgs
        titles = [filename] + titles
        if save_files: save_files = [""] + save_files
    else:
        plt.title(filename)
        plt.imshow(og_img)
        plt.show()
    fig, axis = plt.subplots(1, len(titles), figsize=(12, 7))
    for i, title in enumerate(titles):
        axis[i].set_title(title)
        axis[i].imshow(out_imgs[i], cmap="gray")
        if save_files and save_files[i]: Image.fromarray(out_imgs[i]).save(save_files[i])
    plt.tight_layout()
    plt.show()

def handle_brightness(img:np.ndarray, param:list, save_info:list=None):
    out_imgs, titles, files = [], [], []
    img_name = save_info[0].split('.')[0]
    try: factor = [float(param[0])]
    except: factor = [0.5, -0.5]

    for fa in factor:
        out_imgs.append(adjust_brightness(img, fa))
        if fa < 0:
            oper = "darken"
            titles.append(f"Darken by {-fa * 100}%")
        else:
            oper = "brighten"
            titles.append(f"Brighten by {fa * 100}%")
        if save_info[1]: files.append(f"{img_name}_{oper}_{int(abs(fa) * 100)}{save_info[1]}")
    plot_and_save(save_info[0], img, out_imgs, titles, files)

def handle_contrast(img:np.ndarray, param:list, save_info:list=None):
    out_imgs, titles, files = [], [], []
    img_name = save_info[0].split('.')[0]
    try: factor = [float(param[0])]
    except: factor = [0.5, -0.5]

    for fa in factor:
        out_imgs.append(adjust_contrast(img, fa))
        titles.append(f"Contrast adjusted by {int(fa * 255)}=int({fa}*255)")
        if save_info[1]: files.append(f"{img_name}_contrast_{int(fa * 255)}{save_info[1]}")
    plot_and_save(save_info[0], img, out_imgs, titles, files)

def handle_flipping(img:np.ndarray, param:list, save_info:list=None):
    axis_names = ["vertical", "horizontal"]
    try: axis = int(param[0])
    except: axis = -1
    axis = np.clip(axis, -1, len(axis_names) - 1)
    if axis != -1:
        out_img, file = flip(img, axis), None
        title = f"Flip {axis_names[axis]}ly"
        if save_info[1]:
            img_name = save_info[0].split('.')[0]
            file = [f"{img_name}_flip_{axis_names[0]}{save_info[1]}"]
        plot_and_save(save_info[0], img, [out_img], [title], file)
        return

    out_imgs, titles, files = [], [], []
    img_name = save_info[0].split('.')[0]
    for i, name in enumerate(axis_names):
        out_imgs.append(flip(img, i))
        titles.append(f"Flip {name}ly")
        if save_info[1]: files.append(f"{img_name}_flip_{name}{save_info[1]}")
    plot_and_save(save_info[0], img, out_imgs, titles, files)

def handle_grayscale_sepia(img:np.ndarray, param:list, save_info:list=None):
    try: oper = int(param[0])
    except: oper = -1
    oper = np.clip(oper, -1, 1)
    img_name = save_info[0].split('.')[0]

    def do_sepia():
        out_img, file, title = rgb_to_sepia(img), None, "Sepia"
        if save_info[1]: file = [f"{img_name}_sepia{save_info[1]}"]
        return out_img, title, file

    if oper == 1:
        out_img, title, file = do_sepia()
        plot_and_save(save_info[0], img, [out_img], [title], file)
        return

    grayscale_methods = ["average", "luminosity"]
    def do_grayscale():
        out_imgs, titles, files = [], [], []
        for i, method in enumerate(grayscale_methods):
            out_imgs.append(rgb_to_grayscale(img, i))
            titles.append(f"Grayscale {method}")
            if save_info[1]: files.append(f"{img_name}_grayscale_{method}{save_info[1]}")
        return out_imgs, titles, files

    if oper == 0:
        try: method = int(param[1])
        except: method = -1
        method = np.clip(method, -1, len(grayscale_methods) - 1)
        if method != -1:
            out_img, file = rgb_to_grayscale(img, method), None
            title = f"Grayscale {grayscale_methods[method]}"
            if save_info[1]: file = [f"{img_name}_grayscale_{grayscale_methods[method]}{save_info[1]}"]
            plot_and_save(save_info[0], img, [out_img], [title], file)
            return
        out_imgs, titles, files = do_grayscale()
        plot_and_save(save_info[0], img, out_imgs, titles, files)
        return
    
    out_imgs, titles, files = do_grayscale()
    sep_img, sep_title, sep_file = do_sepia()
    if files: files += sep_file
    plot_and_save(save_info[0], img, out_imgs + [sep_img], titles + [sep_title], files)

def handle_blurring_sharpening(img:np.ndarray, param:list, save_info:list=None):
    try: oper = int(param[0])
    except: oper = -1
    oper = np.clip(oper, -1, 1)
    img_name = save_info[0].split('.')[0]

    def do_blurring(repeat:int=1):
        out_img, file = gaussian_blur_3x3(img), None
        for _ in range(repeat - 1): out_img = gaussian_blur_3x3(out_img)
        title = f"Blur {repeat} time{'s' if repeat != 1 else ''}"
        if save_info[1]: file = [f"{img_name}_blur_{repeat}{save_info[1]}"]
        return out_img, title, file

    def do_sharpening(repeat:int=1):
        out_img, file = sharpen(img), None
        for _ in range(repeat - 1): out_img = sharpen(out_img)
        title = f"Sharpen {repeat} time{'s' if repeat != 1 else ''}"
        if save_info[1]: file = [f"{img_name}_sharpen_{repeat}{save_info[1]}"]
        return out_img, title, file

    if oper == 0:
        try: repeat = int(param[1])
        except: repeat = 0
        out_img, title, file = do_blurring(repeat)
        plot_and_save(save_info[0], img, [out_img], [title], file)
        return
    if oper == 1:
        try: repeat = int(param[1])
        except: repeat = 0
        out_img, title, file = do_sharpening(repeat)
        plot_and_save(save_info[0], img, [out_img], [title], file)
        return

    blur_img, blur_title, blur_file = do_blurring()
    sharp_img, sharp_title, sharp_file = do_sharpening()
    out_imgs, titles, files = [blur_img, sharp_img], [blur_title, sharp_title], None
    if save_info[1]: files = blur_file + sharp_file
    plot_and_save(save_info[0], img, out_imgs, titles, files)

def handle_center_cropping(img:np.ndarray, param:list, save_info:list=None):
    try: ratio = float(param[0])
    except: ratio = 0
    try:
        height, width = param[0].split('x')
        height, width = int(height), int(width)
    except: height = width = 0
    if not ratio and not height and not width: ratio = 0.5
    img_name = save_info[0].split('.')[0]
    if ratio:
        out_img, file = center_crop(img, ratio), None
        title = f"Center crop with {ratio * 100}% dimension"
        if save_info[1]: file = [f"{img_name}_centercrop_{int(ratio * 100)}{save_info[1]}"]
        plot_and_save(save_info[0], img, [out_img], [title], file)
        return
    
    height, width = np.clip(height, 2, None), np.clip(width, 2, None)
    out_img, file = center_crop(img, (height, width)), None
    title = f"Center crop with {width}x{height} dimension"
    if save_info[1]: file = [f"{img_name}_centercrop_{width}x{height}{save_info[1]}"]
    plot_and_save(save_info[0], img, [out_img], [title], file)

def handle_circle_cropping(img:np.ndarray, param:list, save_info:list=None):
    try: radius = float(param[0])
    except: radius = 0
    try:
        y, x = param[1].split('x')
        center = (int(y), int(x))
    except: center = None
    if radius > 1: radius = int(radius)
    out_img, file = circle_crop(img, radius, center), None
    if not radius: rad = "optimal"
    elif radius <= 1: rad = f"{radius * 100}%"
    else: rad = radius
    if not center: center = (int(img.shape[1] / 2), int(img.shape[0] / 2))
    else: center = (center[1], center[0])
    title = f"Circle crop with {rad} radius and center ({center[0]}, {center[1]})"
    if save_info[1]:
        img_name = save_info[0].split('.')[0]
        if radius and radius <= 1: rad = f"{int(radius * 100)}%"
        file = [f"{img_name}_circlecrop_{rad}_{center[0]}x{center[1]}{save_info[1]}"]
    plot_and_save(save_info[0], img, [out_img], [title], file)

def handle_ellip_cropping(img:np.ndarray, param:list, save_info:list=None):
    try:
        rad_y, rad_x = param[0].split('x')
        radii = (abs(float(rad_y)), abs(float(rad_x)))
    except: radii = 0
    try: radius = int(param[0])
    except: radius = 0
    if radii: radius = radii
    try:
        y, x = param[1].split('x')
        center = (int(y), int(x))
    except: center = None
    # out_img, file = cross_ellipses_crop(img, radius, center), None
    # if not radius: rad = "optimal radii"
    # elif radii: rad = f"({int(radii[1])}, {int(radii[0])}) radii"
    # else: rad = f"{radius} radius"
    # if not center: center = (int(img.shape[1] / 2), int(img.shape[0] / 2))
    # else: center = (center[1], center[0])
    # title = f"Cross Ellipses crop with {rad} and center ({center[0]}, {center[1]})"
    # if save_info[1]:
    #     img_name = save_info[0].split('.')[0]
    #     if not radius: rad = "optimal"
    #     elif radii: rad = f"{int(radii[1])}x{int(radii[0])}"
    #     else: rad = f"{radius}"
    #     file = [f"{img_name}_ellipcrop_{rad}_{center[0]}x{center[1]}{save_info[1]}"]
    # plot_and_save(save_info[0], img, [out_img], [title], file)

handlers = [handle_brightness, handle_contrast, handle_flipping,
            handle_grayscale_sepia, handle_blurring_sharpening,
            handle_center_cropping, handle_circle_cropping,
            handle_ellip_cropping]

def handle_all(img:np.ndarray, save_info:list=None):
    for ha in handlers: ha(img, [], save_info)

## Main program handling interfaces, inputs and outputs

**Inputs must follow the format of `<img_file> <function> <parameters>`. Inputs do NOT need to be fully complete.** As long as the `<img_file>` is correct, the `<function>` and **especially `<parameters>` are optional.** The `<parameters>` varies across functions, which is noted below.
- Every input concerning `tuple`s must be in the format of `<Y>x<X>`. E.g., `420x690` means `y=420,x=690` or `height=420,width=690`.
- E.g., `img.jpg 7 0.5 600x400`, `img.png 7 100`, `img.jpg 7`, `img.png 4`, etc.

   0. **All** - Inputting only the `<img_file>` will default to this function. `<parameters>` is NOT required.
   1. **Brightness** - `<factor>`, `float` denoting how much the brightness will be adjusted.
      - E.g., `-0.5` darkens by `50%`, `0.35` brightens by `35%`.
   1. **Contrast** - `<factor>`, `float` denoting how much the contrast will be adjusted.
      - E.g., `-0.5` adjusts by `-127=int(-0.5*255)`, `0.35` adjusts by `89=int(0.35*255)`.
   1. **Flipping** - `<axis>`, `int` representing the direction to flip the image.
      - `0` for vertical, `1` for horizontal, `<blank>` for both.
   1. **Grayscale and Sepia** - `0` for grayscale, `1` for sepia, `<blank>` for both.
      - After `0` for grayscale, `0` for average, `1` for luminosity, `<blank>` for both.
   1. **Blurring and Sharpening** - `0` for blurring, `1` for sharpening, `<blank>` for one execution of each.
      - After `0` or `1`, input an `int` denoting how many times to blur or sharpen.
   1. **Center cropping** - Two methods of inputting.
      - `tuple` of `(height, width)` denoting the dimension of the cropped image. E.g., `420x690`, `727x135`.
      - `float` denoting the ratio between the cropped and the original image. E.g., `0.5` means `50%` of the original image.
   1. **Circle cropping** - Two parameters.
      1. `<radius>` - The domains of this value produce different outcomes.
         - `int (rad > 1)` denoting the radius of the cropped image.
         - `<float> (0 < rad <= 1)` denoting ratio between the cropped and the original image.
         - Leave `<blank>` or `0` to use the smallest distance between the center and image walls.
      1. `<center>`, `tuple (y, x)` denoting the coordinate of the center of the cropped image.
         - Leave `<blank>` to use the center of the original image.
   1. **Cross Ellipses cropping** - Two parameters.
      - *To get the best result, everything should be left `<blank>`.*
      1. `<radius>` - The datatypes of this value produces different outcomes.
         - `tuple (rad_y, rad_x)` denoting the radii of the cropped image.
         - `int` denoting the radius of the cropped image.
         - Leave `<blank>` or `0` to use the optimal dimension.
      1. `<center>` - `tuple (y, x)` representing the coordinate of the center of the cropped image.
         - Leave `<blank>` to use the center of the original image.

In [ ]:
def make_menu(functions:list):
    menu = ""
    for i, func in enumerate(functions):
        menu += f"{i}. {func} "
    menu += "\n"
    return menu

def main():
    functions = ["All", "Brightness", "Contrast", "Flipping", "Grayscale and Sepia",
            "Blurring and Sharpening", "Center cropping", "Circle cropping",
            "Cross Ellipses cropping"]
    outfile_types = [".png", ".pdf"]

    inp = input(f"{make_menu(functions)}| Input img, function, parameters (if required) \n-->")
    filename, *func_param = inp.split(" ")
    try: img = Image.open(filename)
    except:
        print("Invalid file!")
        return
    try: func = np.clip(int(func_param[0]), 0, len(functions) - 1)
    except: func = 0

    outfile_type = input("Save output images as (0 - .png, 1 - .pdf). \
                         Leave blank to opt out of saving. \n--> ")
    try: outfile_type = outfile_types[int(outfile_type)]
    except: outfile_type = 0

    img_arr = np.asarray(img).copy()
    func_param = func_param[1:]
    save_info = [filename, ""]
    if outfile_type: save_info[1] = outfile_type

    if func == 0: handle_all(img_arr, save_info)
    else: handlers[func - 1](img_arr, func_param, save_info)
    img.close()

main()

## References

### 1, 2, 3. Brightness, Contrast, Flipping
- https://www.dfstudios.co.uk/articles/programming/image-programming-algorithms/image-processing-algorithms-part-4-brightness-adjustment/
- https://stackoverflow.com/a/24022126
- https://www.skytowner.com/explore/limiting_array_values_to_a_certain_range_in_numpy#:~:text=To%20limit%20array%20values%20to,provided%20minimum%20and%20maximum%20values.&text=The%20resulting%20clipped%20array%20is%20returned%20and%20the%20original%20array%20remains%20unchanged.
- https://www.dfstudios.co.uk/articles/programming/image-programming-algorithms/image-processing-algorithms-part-5-contrast-adjustment/
- https://numpy.org/doc/stable/reference/generated/numpy.flip.html

### 4. Grayscale and Sepia
- https://www.baeldung.com/cs/convert-rgb-to-grayscale
- https://numpy.org/doc/stable/reference/generated/numpy.matmul.html
- https://dyclassroom.com/image-processing-project/how-to-convert-a-color-image-into-sepia-image
- https://stackoverflow.com/a/10463090

### 5. Blur and Sharpen
- https://en.wikipedia.org/wiki/Kernel_(image_processing)
- https://youtu.be/C_zFhWdM4ic
- https://github.com/pooyakalahroodi/ImageBlurring
- https://stackoverflow.com/questions/40034993/how-to-get-element-wise-matrix-multiplication-hadamard-product-in-numpy
- https://www.geeksforgeeks.org/what-is-three-dots-or-ellipsis-in-python3/

### 6, 7. Center and Circular cropping
- https://stackoverflow.com/a/44874588
- https://towardsdatascience.com/the-little-known-ogrid-function-in-numpy-19ead3bdae40

### Extra. Cross Ellipses cropping
- Will be made publicly available after 2023-July-30.

## Past implementations

- **These codes are retained purely for reference, there are better variants above.**

- [How to convert a color image into sepia image, 2014-01-27](https://dyclassroom.com/image-processing-project/how-to-convert-a-color-image-into-sepia-image) - [Yusuf Shakeel](https://www.youtube.com/@yusufshakeel)

$$ tr = 0.393R + 0.769G + 0.189B \\ tg = 0.349R + 0.686G + 0.168B \\ tb = 0.272R + 0.534G + 0.131B $$

- [What is the general equation of the ellipse that is not in the origin and rotated by an angle?, 2020-12-13](https://math.stackexchange.com/q/434482) - [andikat dennis](https://math.stackexchange.com/users/82597/andikat-dennis)

$$ (\frac{(x-h)\cos(A)+(y-k)\sin(A)}{a})^2 + (\frac{(x-h)\sin(A)-(y-k)\cos(A)}{b})^2 = 1, $$
"where $h,k$ and $a,b$ are the shifts and semi-axis in the $x$ and $y$ directions respectively and $A$ is the angle measured from $x$ axis."

In [ ]:
# def rgb_to_sepia(img:np.ndarray):
#     '''
#     > img: np.ndarray with shape=(height, width, num_channels)
#         Original image as an array
#     < new_img: np.ndarray with shape=(height, width, num_channels)
#     '''
#     rgb_weights = np.array([[0.393, 0.769, 0.189],
#                             [0.349, 0.686, 0.168],
#                             [0.272, 0.534, 0.131]])
#     new_img = img.reshape(img.shape[0] * img.shape[1], img.shape[2]).copy()
#     for i in range(new_img.shape[0]):
#         new_img[i] = np.clip(np.matmul(rgb_weights, new_img[i]), 0, 255).astype(np.uint8)
#     return new_img.reshape(img.shape)

# def create_ellipse_mask(Ys:np.ndarray, Xs:np.ndarray, center:tuple, angle:float, radius:tuple):
#     '''
#     > Ys, Xs: np.ndarrays with shape=(height, 1) and (1, width) respectively
#         Indices of rows and columns accordingly
#     > center: tuple (y, x) representing the coordinate of the center of the cropped image
#     > angle: float representing the angle of rotation of the ellipse
#     > radius: tuple (rad_y, rad_x) representing the radii of the ellipse

#     < new_img: np.ndarray with shape=(dim[0], dim[1], num_channels)
#     '''
#     diff_Xs_center, diff_Ys_center = Xs - center[1], Ys - center[0]
#     sin_angle, cos_angle = np.sin(angle), np.cos(angle)
#     ellipse_Ys = diff_Xs_center * sin_angle - diff_Ys_center * cos_angle
#     ellipse_Xs = diff_Xs_center * cos_angle + diff_Ys_center * sin_angle
#     ellipse = (ellipse_Ys / radius[0])**2 + (ellipse_Xs / radius[1])**2
#     return ellipse <= 1

# def make_menu(functions:list): # The format doesn't work well with ipynb
#     menu = ""
#     ceil_half_len = j = len(functions) // -2 * -1
#     for i in range(ceil_half_len):
#         menu += f"{i}. {functions[i]}\t\t"
#         if j < len(functions):
#             menu += f"{j}. {functions[j]}\n"
#             j += 1
#         else: menu += "\n"
#     return menu

In [ ]:
# def main():
#     files = ['hcmus.jpg', 'kino.jpg', 'kino_grayscale.png','lenna.png', 'world_pulse.jpg']
#     filename = files[4]
#     try: img = Image.open(filename)
#     except:
#         print("Invalid file!")
#         return
#     img_arr = np.asarray(img).copy()
#     func = cross_ellipses_crop

#     # factor_1 = 0.5
#     # factor_2 = -0.5
#     output_1 = func(img_arr)
#     #output_2 = func(img_arr)
#     #for _ in range(9): output_2 = func(output_2)
#     fig, axis = plt.subplots(1, 2, figsize=(12, 7))
#     axis[0].set_title(filename)
#     axis[0].imshow(img, cmap='gray')
#     #axis[1].set_title(f"Cross Ellipses crop with optimal radii and center ({int(img_arr.shape[1]/2)},{int(img_arr.shape[0]/2)})")
#     axis[1].set_title(f"output")
#     axis[1].imshow(output_1, cmap='gray')
#     # axis[2].set_title("Blur 10")
#     # axis[2].imshow(output_2, cmap="gray")
#     plt.tight_layout()
#     #plt.savefig("plot.pdf", format="pdf", bbox_inches="tight")
#     plt.show()
#     img.close()
# main()